In [3]:
import pandas as pd

# Load your existing pivot file
df = pd.read_csv("financials_Q1_Q2_2025.csv")

# Fix duplicate or missing values
# For Net Income: if one tag is missing, use the other
df['Net_Income_Final'] = df[['Net_Income']].bfill(axis=1).iloc[:, 0]

# For Revenue: if one tag is missing, fill with alternative columns if available
# Add similar logic if you had multiple revenue tags
df['Revenue_Final'] = df['Revenue']

# Drop old columns and keep only final cleaned values
df_clean = df[['name', 'period', 'EPS_Basic', 'EPS_Diluted', 'Net_Income_Final', 'Revenue_Final']]
df_clean = df_clean.rename(columns={
    'Net_Income_Final': 'Net_Income',
    'Revenue_Final': 'Revenue'
})

# Save final cleaned dataset
df_clean.to_csv("financials_cleaned_Q1_Q2_2025.csv", index=False)
print("✅ Cleaned data saved as: financials_cleaned_Q1_Q2_2025.csv")


✅ Cleaned data saved as: financials_cleaned_Q1_Q2_2025.csv


In [8]:
# === Set up ===
folders = ["2025q1", "2025q2"]
sub_files = [f"{folder}/sub.txt" for folder in folders]
num_files = [f"{folder}/num.txt" for folder in folders]

# Companies to extract
target_names = [
    "ALPHABET INC.", "BLACKSTONE INC.", "DELL TECHNOLOGIES INC.",
    "GOLDMAN SACHS GROUP INC", "WARNER BROS. DISCOVERY, INC."
]

# XBRL Tags for Financial Metrics
revenue_tags = [
    'Revenues', 'RevenuesNet', 'SalesRevenueNet', 'SalesRevenueServicesNet'
]
net_income_tags = ['NetIncomeLoss', 'ProfitLoss']
eps_tags = ['EarningsPerShareBasic', 'EarningsPerShareDiluted']

all_tags = revenue_tags + net_income_tags + eps_tags

# === Extract + Merge ===
combined = []

for sub_file, num_file in zip(sub_files, num_files):
    sub = pd.read_csv(sub_file, sep='\t', low_memory=False)
    num = pd.read_csv(num_file, sep='\t', low_memory=False)

    # Filter to target companies
    sub = sub[sub['name'].isin(target_names)]
    num = num[num['tag'].isin(all_tags)]

    # Merge to get company + period info
    merged = pd.merge(num, sub[['adsh', 'name', 'period']], on='adsh')
    filtered = merged[['name', 'period', 'tag', 'value']]
    combined.append(filtered)

# Combine all quarters
all_df = pd.concat(combined, ignore_index=True)

# === Pivot ===
pivot = all_df.pivot_table(index=['name', 'period'], columns='tag', values='value', aggfunc='first').reset_index()

# === Safe fallback for tag groups ===
available_revenue_tags = [tag for tag in revenue_tags if tag in pivot.columns]
available_net_income_tags = [tag for tag in net_income_tags if tag in pivot.columns]

pivot['Revenue'] = pivot[available_revenue_tags].bfill(axis=1).iloc[:, 0] if available_revenue_tags else None
pivot['Net_Income'] = pivot[available_net_income_tags].bfill(axis=1).iloc[:, 0] if available_net_income_tags else None
pivot['EPS_Basic'] = pivot['EarningsPerShareBasic'] if 'EarningsPerShareBasic' in pivot.columns else None
pivot['EPS_Diluted'] = pivot['EarningsPerShareDiluted'] if 'EarningsPerShareDiluted' in pivot.columns else None

# === Final Cleaned Output ===
final_df = pivot[['name', 'period', 'EPS_Basic', 'EPS_Diluted', 'Net_Income', 'Revenue']]
final_df.to_csv("final_clean_financials_Q1_Q2_2025.csv", index=False)
print("✅ Cleaned data saved as: final_clean_financials_Q1_Q2_2025.csv")


✅ Cleaned data saved as: final_clean_financials_Q1_Q2_2025.csv
